In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import re
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import matplotlib.pyplot as plt


### Clear day-ahead load

In [6]:
# Specify the directory you want to use
directory = './DAM/Load'

dataframes = []  # list to store dataframes
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        df = pd.read_csv(os.path.join(directory, filename), encoding='ISO-8859-1')
        dataframes.append(df)
combined_df = pd.concat(dataframes, ignore_index=True)

In [7]:
values = ['CA ISO-TAC', 'MWD-TAC', 'PGE-TAC','SCE-TAC','SDGE-TAC','VEA-TAC']
df_selected = combined_df[combined_df['TAC_AREA_NAME'].isin(values)]
df_sorted = df_selected.sort_values(by=['OPR_DT', 'OPR_HR',]).reset_index(drop=True)

In [8]:
CAISO = df_sorted[df_sorted['TAC_AREA_NAME']=='CA ISO-TAC'][['INTERVALSTARTTIME_GMT','MW']].iloc[24:8784].reset_index(drop=True)
Zone1 = df_sorted[df_sorted['TAC_AREA_NAME']=='PGE-TAC'][['INTERVALSTARTTIME_GMT','MW']].iloc[24:8784].reset_index(drop=True)
Zone2_values = ['MWD-TAC','SCE-TAC','VEA-TAC']
Zone2_sep = df_sorted[df_sorted['TAC_AREA_NAME'].isin(Zone2_values)]
Zone2 = Zone2_sep.groupby(['INTERVALSTARTTIME_GMT'])['MW'].sum().iloc[24:8784].reset_index()
Zone3 = df_sorted[df_sorted['TAC_AREA_NAME']=='SDGE-TAC'][['INTERVALSTARTTIME_GMT','MW']].iloc[24:8784].reset_index(drop=True)

In [9]:
train_pre = pd.concat([CAISO['INTERVALSTARTTIME_GMT'], CAISO['MW'], Zone1['MW'], Zone2['MW'], Zone3['MW']], axis=1)
train_pre.columns = ['INTERVALSTARTTIME_GMT', 'CAISO', 'Zone1', 'Zone2', 'Zone3']
train_pre['CAISO'] = train_pre[['Zone1', 'Zone2', 'Zone3']].sum(axis=1)
train_pre['INTERVALSTARTTIME_GMT'] = pd.to_datetime(train_pre['INTERVALSTARTTIME_GMT'])
train_pre.set_index('INTERVALSTARTTIME_GMT', inplace=True)

In [10]:
train_pre

CAISO     Zone1     Zone2   Zone3
INTERVALSTARTTIME_GMT                                          
2022-01-01 08:00:00+00:00  22630.73  10247.46  10307.27  2076.0
2022-01-01 09:00:00+00:00  21765.85   9876.01   9899.84  1990.0
2022-01-01 10:00:00+00:00  21104.63   9598.49   9587.14  1919.0
2022-01-01 11:00:00+00:00  20740.12   9453.62   9417.50  1869.0
2022-01-01 12:00:00+00:00  20710.33   9498.25   9353.08  1859.0
...                             ...       ...       ...     ...
2023-01-01 03:00:00+00:00  25297.50  11625.42  11119.08  2553.0
2023-01-01 04:00:00+00:00  24463.21  11225.39  10761.82  2476.0
2023-01-01 05:00:00+00:00  23822.35  10897.86  10500.49  2424.0
2023-01-01 06:00:00+00:00  23038.80  10464.78  10262.02  2312.0
2023-01-01 07:00:00+00:00  22090.16   9988.81   9902.35  2199.0

[8760 rows x 4 columns]

In [11]:
DAMLoad = train_pre.resample('5T').ffill()

C:\Users\wenmi\AppData\Local\Temp\ipykernel_18412\3571447211.py:1: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  DAMLoad = train_pre.resample('5T').ffill()


In [12]:
train_pre['CAISO'].max()

51317.22

### K-Means training

In [13]:
reshaped_data = train_pre[['CAISO', 'Zone1', 'Zone2', 'Zone3']].values.reshape(-1, 24*4)/max(train_pre['CAISO'])  # reshapes data to have 365 data points each of 24*4=96 features

kmeans = KMeans(n_clusters= 8, random_state=0).fit(reshaped_data)

labels = kmeans.labels_  # the label of the cluster each data point belongs to

C:\Users\wenmi\.conda\envs\STESTS\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


In [14]:
# Generate some data
x1 = np.linspace(0, 365, 365)
y1 = labels

# Create a figure and axis
fig, ax = plt.subplots()

# Plot the data
ax.plot(x1, y1)

# Show the plot
plt.show()

### Clear real-time load

In [15]:
# Specify the directory you want to use
directory = './RTD/Load'

dataframes = []  # list to store dataframes
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        df = pd.read_csv(os.path.join(directory, filename))
        dataframes.append(df)
combined_df = pd.concat(dataframes, ignore_index=True)

In [16]:
combined_df = combined_df.sort_values(by=['INTERVALSTARTTIME_GMT', 'TAC_AREA_NAME']).reset_index(drop=True)

In [17]:
combined_df

INTERVALSTARTTIME_GMT        INTERVALENDTIME_GMT  LOAD_TYPE  \
0        2021-12-31T08:00:00-00:00  2021-12-31T08:05:00-00:00          0   
1        2021-12-31T08:00:00-00:00  2021-12-31T08:05:00-00:00          0   
2        2021-12-31T08:00:00-00:00  2021-12-31T08:05:00-00:00          0   
3        2021-12-31T08:00:00-00:00  2021-12-31T08:05:00-00:00          0   
4        2021-12-31T08:00:00-00:00  2021-12-31T08:05:00-00:00          0   
...                            ...                        ...        ...   
3130566  2023-01-01T07:55:00-00:00  2023-01-01T08:00:00-00:00          0   
3130567  2023-01-01T07:55:00-00:00  2023-01-01T08:00:00-00:00          0   
3130568  2023-01-01T07:55:00-00:00  2023-01-01T08:00:00-00:00          0   
3130569  2023-01-01T07:55:00-00:00  2023-01-01T08:00:00-00:00          0   
3130570  2023-01-01T07:55:00-00:00  2023-01-01T08:00:00-00:00          0   

             OPR_DT  OPR_HR  OPR_INTERVAL MARKET_RUN_ID TAC_AREA_NAME  \
0        2021-12-31       1             1           RTM          AZPS   
1        2021-12-31       1             1           RTM          BANC   
2        2021-12-31       1             1           RTM       BANCMID   
3        2021-12-31       1             1           RTM      BANCRDNG   
4        2021-12-31       1             1           RTM      BANCRSVL   
...             ...     ...           ...           ...           ...   
3130566  2022-12-31      24            12           RTM           SRP   
3130567  2022-12-31      24            12           RTM          TEPC   
3130568  2022-12-31      24            12           RTM          TIDC   
3130569  2022-12-31      24            12           RTM          TPWR   
3130570  2022-12-31      24            12           RTM       VEA-TAC   

                          LABEL     XML_DATA_ITEM  POS       MW  \
0        RTM 5Min Load Forecast  SYS_FCST_5MIN_MW  5.8  2858.05   
1        RTM 5Min Load Forecast  SYS_FCST_5MIN_MW  5.8  1670.37   
2        RTM 5Min Load Forecast  SYS_FCST_5MIN_MW  5.8   240.14   
3        RTM 5Min Load Forecast  SYS_FCST_5MIN_MW  5.8   101.72   
4        RTM 5Min Load Forecast  SYS_FCST_5MIN_MW  5.8   109.24   
...                         ...               ...  ...      ...   
3130566  RTM 5Min Load Forecast  SYS_FCST_5MIN_MW  5.8  2676.27   
3130567  RTM 5Min Load Forecast  SYS_FCST_5MIN_MW  5.8  1069.41   
3130568  RTM 5Min Load Forecast  SYS_FCST_5MIN_MW  5.8   240.64   
3130569  RTM 5Min Load Forecast  SYS_FCST_5MIN_MW  5.8   490.33   
3130570  RTM 5Min Load Forecast  SYS_FCST_5MIN_MW  5.5    64.41   

        EXECUTION_TYPE  GROUP  
0                  RTD      1  
1                  RTD      2  
2                  RTD      3  
3                  RTD      4  
4                  RTD      5  
...                ...    ...  
3130566            RTD    926  
3130567            RTD    927  
3130568            RTD    928  
3130569            RTD    929  
3130570            RTD    930  

[3130571 rows x 14 columns]

In [18]:
values = ['CA ISO-TAC', 'MWD-TAC', 'PGE-TAC','SCE-TAC','SDGE-TAC','VEA-TAC']
df_selected = combined_df[combined_df['TAC_AREA_NAME'].isin(values)]
df_selected = df_selected[df_selected['XML_DATA_ITEM']=='SYS_FCST_5MIN_MW'].reset_index(drop=True)[['INTERVALSTARTTIME_GMT', 'TAC_AREA_NAME', 'MW']]

In [19]:
df_selected['INTERVALSTARTTIME_GMT'] = pd.to_datetime(df_selected['INTERVALSTARTTIME_GMT'])
df_pivot = df_selected.pivot(index='INTERVALSTARTTIME_GMT', columns='TAC_AREA_NAME', values='MW')
df_resampled = df_pivot.resample('5T').asfreq()
df_interpolated = df_resampled.interpolate(method='linear')

C:\Users\wenmi\AppData\Local\Temp\ipykernel_18412\2980505800.py:3: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_resampled = df_pivot.resample('5T').asfreq()


In [20]:
CAISO = df_interpolated['CA ISO-TAC'][288:105397]
Zone1 = df_interpolated['PGE-TAC'][288:105397]
Zone2 = (df_interpolated['MWD-TAC']+df_interpolated['SCE-TAC']+df_interpolated['VEA-TAC'])[288:105397]
Zone3 = df_interpolated['SDGE-TAC'][288:105397]
RTDLoad = pd.concat([CAISO, Zone1, Zone2, Zone3], axis=1)
RTDLoad.columns = ['CAISO', 'Zone1', 'Zone2', 'Zone3']
RTDLoad['CAISO'] = RTDLoad[['Zone1', 'Zone2', 'Zone3']].sum(axis=1)


### RTD load noise

In [21]:
RTDLoadMAE = sum((abs((RTDLoad['CAISO'].values - DAMLoad['CAISO'].values))/RTDLoad['CAISO'].values))/105109

RTDLoadMAE

0.0750120335207664

In [22]:
RTDLoadNoise = (DAMLoad - RTDLoad)/max(DAMLoad['CAISO'])

In [23]:
RTDLoadNoise['CAISO'].max()

0.14039069146769848

In [24]:
### find closest date to each cluster centroid
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, reshaped_data)
noise_list = []
for i in closest:
    selected_rows = RTDLoadNoise.iloc[288*i:288*(i+1)].values
    noise_list.append(selected_rows)
noise_list = np.stack(noise_list)
# j = 0
# for i in closest:
#     selected_rows = RTDLoadNoise.loc[RTDLoadNoise.index.date == train_pre.index.date[24*i]]
#     selected_rows.to_csv(f"./noise/cluster_{j}.csv")
#     j=j+1

In [25]:
noise_list[:, :, 0].shape

(8, 288)

In [26]:
noise_list[:, :, 0].min()

-0.013322038878957157

### ADS day-ahead load prediction

In [36]:
ADSLoad = pd.read_csv("./Processed Data/2022/Load.csv")[['1','2','3']]
ADSLoad['CAISO'] = ADSLoad[['1', '2', '3']].sum(axis=1)
ADSLoad = ADSLoad[['CAISO', '1', '2', '3']]
reshaped_ADSLoad = ADSLoad.values.reshape(-1, 24*4)/max(ADSLoad['CAISO'])  # reshapes data to have 365 data points each of 24*4=96 features

In [37]:
ADSLoad['CAISO'].max()/train_pre['CAISO'].max()

1.2262549101451716

In [38]:
ADSLoad['CAISO'].mean()/train_pre['CAISO'].mean()

1.4165691671861582

In [39]:
ADSLoad_C = pd.read_csv("./Processed Data/2022/Load.csv")

In [40]:
ADSLoad_C = ADSLoad_C.iloc[:,1:8]/(ADSLoad['CAISO'].max()/train_pre['CAISO'].max())
ADSLoad_C.to_csv('Processed Data/2022/Load_C.csv')

In [41]:
ADSLoad_C

1             2            3             4             5  \
0     10560.134677  11487.856141  1603.255558  18480.659945  17909.816155   
1      9764.093828  10636.344770  1484.193853  17656.198414  17459.665052   
2      9174.450522  10130.918863  1350.453308  17299.013300  17400.134200   
3      8817.452970   9767.682805  1240.362006  17174.243158  17454.772106   
4      8744.054691   9627.255232  1204.480396  17429.491881  18157.725458   
...            ...           ...          ...           ...           ...   
8779  15243.043551  16165.113661  2783.271220  21563.216409  23036.808877   
8780  14946.561966  15979.703598  2733.526261  20970.354359  22496.138259   
8781  14523.675178  15736.467059  2611.202592  20326.116368  21411.535059   
8782  13795.956175  15126.602019  2457.074769  19408.688849  20483.506155   
8783  12677.081145  14207.535363  2207.534484  18260.477340  19657.413643   

                 6            7  
0     11461.781628  1509.474078  
1     11169.835804  1447.496752  
2     11037.726241  1414.877107  
3     11066.268431  1385.519427  
4     11302.760858  1377.364515  
...            ...          ...  
8779  15140.631729  1716.608825  
8780  15200.162581  1699.483511  
8781  15010.153148  1649.738552  
8782  14484.161371  1578.790824  
8783  13850.524764  1489.086800  

[8784 rows x 7 columns]

In [42]:
ADSLoadC = ADSLoad_C[['1','2','3']]
ADSLoadC['CAISO'] = ADSLoadC[['1', '2', '3']].sum(axis=1)
ADSLoadC = ADSLoadC[['CAISO', '1', '2', '3']]

C:\Users\wenmi\AppData\Local\Temp\ipykernel_18412\1359669237.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ADSLoadC['CAISO'] = ADSLoadC[['1', '2', '3']].sum(axis=1)


In [43]:
reshaped_ADSLoad

array([[0.46088324, 0.2057815 , 0.22385967, ..., 0.22390465, 0.24455603,
        0.03650204],
       [0.45385911, 0.20068848, 0.22286616, ..., 0.22230719, 0.2421857 ,
        0.03774155],
       [0.45247412, 0.20139492, 0.21880421, ..., 0.22754926, 0.22025985,
        0.03550089],
       ...,
       [0.442139  , 0.19612003, 0.21234556, ..., 0.23688625, 0.26010888,
        0.04060196],
       [0.48440974, 0.21187809, 0.23812725, ..., 0.24415025, 0.26999463,
        0.04066553],
       [0.49644231, 0.21766192, 0.24458257, ..., 0.24703367, 0.27685707,
        0.04301742]])

In [44]:
predictions = kmeans.predict(reshaped_ADSLoad)
predictions

array([2, 2, 5, 5, 3, 3, 3, 2, 2, 2, 5, 2, 2, 2, 2, 2, 4, 4, 3, 3, 3, 3,
       3, 4, 4, 3, 3, 3, 2, 2, 5, 5, 2, 2, 2, 2, 2, 5, 6, 2, 2, 2, 2, 2,
       4, 6, 2, 3, 3, 3, 3, 4, 4, 2, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 5,
       6, 2, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 5, 6,
       2, 2, 2, 2, 2, 4, 4, 3, 3, 3, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 5, 3,
       3, 3, 3, 2, 4, 4, 2, 2, 2, 2, 3, 3, 2, 3, 0, 0, 0, 3, 4, 4, 2, 2,
       3, 3, 2, 4, 5, 2, 2, 3, 3, 2, 4, 5, 2, 0, 3, 3, 3, 3, 0, 0, 0, 0,
       0, 0, 3, 3, 0, 7, 7, 7, 0, 3, 2, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0,
       0, 0, 3, 0, 0, 2, 0, 7, 7, 0, 7, 7, 7, 7, 7, 0, 0, 7, 7, 7, 7, 7,
       0, 0, 7, 7, 7, 7, 7, 0, 0, 7, 7, 7, 7, 7, 7, 0, 7, 7, 7, 7, 7, 7,
       0, 7, 7, 7, 7, 7, 7, 0, 7, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0,
       0, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0, 3, 0, 0, 0, 7, 7, 0, 3, 0,
       0, 7, 7, 0, 3, 3, 0, 0, 0, 0, 3, 2, 2, 3, 3, 3, 3, 3, 2, 5, 3, 3,
       3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 2, 2, 3, 3, 3,

In [45]:
# Generate some data
x2 = np.linspace(0, 366, 366)
y2 = predictions

# Create a figure and axis
fig, ax = plt.subplots()

# Plot the data
ax.plot(x1, y1)

# Plot the data
ax.plot(x2, y2)

# Show the plot
plt.show()

### Apply noise to ADS load base on prediciton

In [46]:
ADSLoad['CAISO'].mean()

35579.44421083788

In [47]:
ADSLoadC['CAISO'].mean()

29014.721096305962

In [48]:
ADSLoad['CAISO'].mean()/ADSLoadC['CAISO'].mean()

1.2262549101451714

In [49]:
ADSLoad['CAISO'].max()/ADSLoadC['CAISO'].max()

1.2262549101451716

In [50]:
max(ADSLoad['CAISO'])

62927.993

In [51]:
noise_list_ADS = max(ADSLoad['CAISO'])*noise_list
noise_list_ADS[1]

array([[1115.6835049 ,  586.50546097,  646.40801333, -117.22996941],
       [1194.50717052,  618.89085315,  680.80446356, -105.18814619],
       [1266.4024959 ,  657.89802184,  715.59331536, -107.0888413 ],
       ...,
       [1624.7509683 ,  749.41342579,  857.66720925,   17.67033326],
       [1724.87468171,  802.11786182,  899.51928934,   23.23753055],
       [1795.45791434,  836.07286029,  934.90900604,   24.47604801]])

In [52]:
noise_list_ADSC = max(ADSLoadC['CAISO'])*noise_list
noise_list_ADSC[1]

array([[ 909.83,  478.29,  527.14,  -95.6 ],
       [ 974.11,  504.7 ,  555.19,  -85.78],
       [1032.74,  536.51,  583.56,  -87.33],
       ...,
       [1324.97,  611.14,  699.42,   14.41],
       [1406.62,  654.12,  733.55,   18.95],
       [1464.18,  681.81,  762.41,   19.96]])

In [53]:
ADSLoad_repeated = ADSLoad.loc[ADSLoad.index.repeat(12)].reset_index(drop=True)

In [54]:
ADSLoadC_repeated = ADSLoadC.loc[ADSLoad.index.repeat(12)].reset_index(drop=True)

In [55]:
-noise_list_ADSC[predictions[0]]+ADSLoadC_repeated[0:288].values

array([[23276.09637631, 10327.34467744, 11278.11614105,  1670.63555782],
       [23187.55637631, 10290.15467744, 11232.46614105,  1664.93555782],
       [23098.23637631, 10251.60467744, 11189.55614105,  1657.07555782],
       ...,
       [24338.31257861, 10714.43398094, 11756.63547645,  1867.24312122],
       [24226.68257861, 10667.03398094, 11701.53547645,  1858.11312122],
       [24125.11257861, 10624.14398094, 11650.34547645,  1850.62312122]])

In [56]:
ADSRTDLoad = np.empty((0, 4))
for i in range(len(reshaped_ADSLoad)):
    adjust = -noise_list_ADS[predictions[i]]+ADSLoad_repeated[i*288:i*288+288].values
    ADSRTDLoad = np.vstack((ADSRTDLoad, adjust))

In [57]:
ADSRTDLoadC = np.empty((0, 4))
for i in range(len(reshaped_ADSLoad)):
    adjustC = -noise_list_ADSC[predictions[i]]+ADSLoadC_repeated[i*288:i*288+288].values
    ADSRTDLoadC = np.vstack((ADSRTDLoadC, adjustC))

In [58]:
len(ADSRTDLoadC)

105408

In [60]:
ADSLoad_rest = pd.read_csv("./Processed Data/2022/Load.csv")[['4','5','6','7']]
new_index = np.arange(ADSLoad_rest.index.min(), ADSLoad_rest.index.max() + 1, 1/12)
ADSLoad_rest_reindexed = ADSLoad_rest.reindex(new_index)
ADSLoad_rest_interpolated = ADSLoad_rest_reindexed.interpolate(method='linear')
ADSLoad_rest_interpolated.values

array([[22662.        , 21962.        , 14055.066     ,  1851.        ],
       [22577.75      , 21916.        , 14025.23266667,  1844.66666667],
       [22493.5       , 21870.        , 13995.39933333,  1838.33333333],
       ...,
       [22392.        , 24105.        , 16984.274     ,  1826.        ],
       [22392.        , 24105.        , 16984.274     ,  1826.        ],
       [22392.        , 24105.        , 16984.274     ,  1826.        ]])

In [61]:
ADSLoadC_rest = pd.read_csv("./Processed Data/2022/Load_C.csv")[['4','5','6','7']]
new_index = np.arange(ADSLoadC_rest.index.min(), ADSLoadC_rest.index.max() + 1, 1/12)
ADSLoadC_rest_reindexed = ADSLoadC_rest.reindex(new_index)
ADSLoadC_rest_interpolated = ADSLoadC_rest_reindexed.interpolate(method='linear')
ADSLoadC_rest_interpolated.values

array([[18480.65994477, 17909.81615511, 11461.78162772,  1509.47407809],
       [18411.95481723, 17872.30356322, 11437.45280906,  1504.30930095],
       [18343.24968969, 17834.79097132, 11413.12399041,  1499.14452381],
       ...,
       [18260.47734019, 19657.41364261, 13850.52476405,  1489.08679989],
       [18260.47734019, 19657.41364261, 13850.52476405,  1489.08679989],
       [18260.47734019, 19657.41364261, 13850.52476405,  1489.08679989]])

In [62]:
ADSLoad_RTD = np.hstack((ADSRTDLoad, ADSLoad_rest_interpolated))
np.savetxt("realtimeload_fixed.csv", ADSLoad_RTD, delimiter=",")


In [63]:
ADSLoadC_RTD = np.hstack((ADSRTDLoadC, ADSLoadC_rest_interpolated))
np.savetxt("realtimeload_C_fixed.csv", ADSLoadC_RTD, delimiter=",")

In [64]:
ADSRTDLoadC

array([[23276.09637631, 10327.34467744, 11278.11614105,  1670.63555782],
       [23187.55637631, 10290.15467744, 11232.46614105,  1664.93555782],
       [23098.23637631, 10251.60467744, 11189.55614105,  1657.07555782],
       ...,
       [27537.38099149, 11842.9911447 , 13517.72536305,  2176.66448374],
       [27409.99099149, 11789.8111447 , 13453.55536305,  2166.62448374],
       [27294.93099149, 11743.3011447 , 13393.48536305,  2158.14448374]])

In [65]:
df_interpolated

TAC_AREA_NAME              CA ISO-TAC  MWD-TAC  PGE-TAC  SCE-TAC  SDGE-TAC  \
INTERVALSTARTTIME_GMT                                                        
2021-12-31 08:00:00+00:00    22062.04   212.02  9385.72  9336.17   2002.17   
2021-12-31 08:05:00+00:00    21971.23   211.99  9347.57  9292.39   1996.91   
2021-12-31 08:10:00+00:00    21877.07   211.80  9305.30  9250.18   1992.05   
2021-12-31 08:15:00+00:00    21781.70   211.40  9258.01  9210.23   1987.69   
2021-12-31 08:20:00+00:00    21672.74   210.66  9202.12  9171.41   1977.69   
...                               ...      ...      ...      ...       ...   
2023-01-01 07:35:00+00:00    20993.08   206.81  8771.84  8897.09   2051.90   
2023-01-01 07:40:00+00:00    20941.46   206.81  8756.46  8867.53   2049.83   
2023-01-01 07:45:00+00:00    20734.96   206.82  8604.44  8807.64   2054.20   
2023-01-01 07:50:00+00:00    20673.72   206.79  8574.65  8779.28   2052.77   
2023-01-01 07:55:00+00:00    20624.15   206.70  8558.20  8748.99   2052.95   

TAC_AREA_NAME              VEA-TAC  
INTERVALSTARTTIME_GMT               
2021-12-31 08:00:00+00:00    70.63  
2021-12-31 08:05:00+00:00    70.80  
2021-12-31 08:10:00+00:00    70.25  
2021-12-31 08:15:00+00:00    70.47  
2021-12-31 08:20:00+00:00    69.99  
...                            ...  
2023-01-01 07:35:00+00:00    63.99  
2023-01-01 07:40:00+00:00    63.81  
2023-01-01 07:45:00+00:00    63.85  
2023-01-01 07:50:00+00:00    64.48  
2023-01-01 07:55:00+00:00    64.41  

[105408 rows x 6 columns]

In [66]:
ADSLoadC_rest_interpolated

4             5             6            7
0.000000     18480.659945  17909.816155  11461.781628  1509.474078
0.083333     18411.954817  17872.303563  11437.452809  1504.309301
0.166667     18343.249690  17834.790971  11413.123990  1499.144524
0.250000     18274.544562  17797.278379  11388.795172  1493.979747
0.333333     18205.839435  17759.765788  11364.466353  1488.814970
...                   ...           ...           ...          ...
8783.583333  18260.477340  19657.413643  13850.524764  1489.086800
8783.666667  18260.477340  19657.413643  13850.524764  1489.086800
8783.750000  18260.477340  19657.413643  13850.524764  1489.086800
8783.833333  18260.477340  19657.413643  13850.524764  1489.086800
8783.916667  18260.477340  19657.413643  13850.524764  1489.086800

[105408 rows x 4 columns]